In [ ]:
%matplotlib inline

import os
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=gpu'

import numpy as np
import os
#from matplotlib import pyplot as plt
#import matplotlib.gridspec as gridspec


#import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec

import gzip
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import numpy as np
#from load import mnist

#import Plots
import pickle
srng = RandomStreams()

f = open("costs.txt", 'w')
f.write("Starting...\n")
f.close()

# writes cost to txt file
def write(str):
    f = open("costs.txt", 'a')
    f.write(str)
    f.write("\n")
    f.close()
    
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def init_biases(n_out):
    return theano.shared(floatX(np.zeros(n_out)))

def plotter(samples, predictions, img_x, idx):
    #plot_all_filters(Ws, idx)
    shp = (samples.shape[0], 1, img_x, img_x)
    samples = samples.reshape(shp)
    predictions = predictions.reshape(shp)
    plot_predictions_grid(samples, predictions, idx, shp, 'preds')
    return

def rectify(X):
    return T.maximum(X, 0.)

def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    return updates

def dropout(X, p=0.):
    if p > 0:
        retain_prob = 1 - p
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
        X /= retain_prob
    return X

# Gaussian prior
def model(X, w1, b1, wmu, bmu, wsigma, bsigma, w2, b2, w3, b3, e):
    h1 = rectify(T.dot(X, w1) + b1) # encoding
    mu = T.dot(h1, wmu) + bmu # using encoding to parameterize mu
    log_sigma = 0.5 * (T.dot(h1, wsigma) + bsigma) # using enc to param sigma
    z = mu + T.exp(log_sigma) * e 
    h2 = rectify(T.dot(z, w2) + b2) # decoder
    out = rectify(T.dot(h2, w3) + b3) # decoder
    return mu, log_sigma, z, out
#need mu, log_sigma, z, wmu/bmu, wsigma/bsigma
#X is input, but you can input unif random X

def generate(Z, w2, b2, w3, b3):
    h2 = rectify(T.dot(Z, w2) + b2)
    out = rectify(T.dot(h2, w3) + b3)
    return out

    
def get_params(code_size):
    n_hidden1 = 600
    n_code = code_size
    n_hidden2 = 600
    w1 = init_weights((784, n_hidden1))
    b1 = init_biases(n_hidden1)
    wmu = init_weights((n_hidden1, n_code))
    bmu = init_biases(n_code)
    wsigma = init_weights((n_hidden1, n_code))
    bsigma = init_biases(n_code)
    w2 = init_weights((n_code, n_hidden2))
    b2 = init_biases(n_hidden2)
    w3 = init_weights((n_hidden2, 784))
    b3 = init_biases(784)
    return w1, b1, wmu, bmu, wsigma, bsigma, w2, b2, w3, b3

def one_hot(x,n):
	if type(x) == list:
		x = np.array(x)
	x = x.flatten()
	o_h = np.zeros((len(x),n))
	o_h[np.arange(len(x)),x] = 1
	return o_h


#def load_data(dataset):
def mnist(ntrain=60000,ntest=10000,onehot=True):
    ''' Loads the dataset
    :type dataset: string
    :param dataset: the path to the dataset (here MNIST)
    '''
    path = "/data/fs4/datasets/mnist/mnist.pkl.gz"
    f = gzip.open(path, 'rb')
    train_set, valid_set, test_set = pickle.load(f)
    f.close()
    # train_set, valid_set, test_set format: tuple(input, target)
    # input is an numpy.ndarray of 2 dimensions (a matrix)
    # which row's correspond to an example. target is a
    # numpy.ndarray of 1 dimensions (vector)) that have the same length as
    # the number of rows in the input. It should give the target
    # target to the example with the same index in the input.
    print("Done.")
    #return (train_set, valid_set, test_set)
    if onehot:
        trY = one_hot(train_set[1], 10)
        teY = one_hot(valid_set[1], 10)
    else:
        trY = np.asarray(train_set[1])
        teY = np.asarray(valid_set[1])
    trX = train_set[0]
    teX = valid_set[0]
    print(trX.shape)
    trX = trX.reshape(trX.shape[0], 1, 28, 28)
    teX = teX.reshape(teX.shape[0], 1, 28, 28)
    return trX, trY, teX, teY, 1, 28




import scipy
import theano
def caltech(onehot=True, num_imgs_to_load=1000):
    path = "data-caltech/101_ObjectCategories/"

    # load file names
    fnames = []
    cats = {}
    for path, subdirs, files in os.walk(path):
        if len(files) > 0:
            cats[path] = []
            #print(path)
        for name in files:
            fnames.append(os.path.join(path, name))
            cats[path].append(os.path.join(path, name))


    # load images
    i=0
    for k in cats.keys():
        cats[i] = cats[k]
        del cats[k]
        i += 1
    #print(cats.keys())
    
    pics = []
    imgshape = []
    #rand = np.array(fnames)
    count = 0
    # for img in fnames:
    #     if count >= num_imgs_to_load:
    #         break
    #     count += 1
    #     pic = scipy.misc.imread(img)
    #     if len(pic.shape)==3:
    #         if pic.shape[0] >= 220 and pic.shape[1] >= 220:
    #             pic = pic[20:20+200, 20:20+200, :]
    #             pic = scipy.misc.imresize(pic, size=(60,60))
    #             imgshape = pic.shape
    #             pics.append(pic)
    #             labels.append()
    
    labels = []
    for label, vals in cats.items():
        for val in vals:
            if count >= num_imgs_to_load:
                break
            count += 1
            pic = scipy.misc.imread(val)
            if len(pic.shape)==3:
                if pic.shape[0] >= 220 and pic.shape[1] >= 220:
                    pic = pic[20:20+200, 20:20+200, :]
                    pic = scipy.misc.imresize(pic, size=(60,60))
                    imgshape = pic.shape
                    pics.append(pic)
                    labels.append(label)
    
        
    X = np.array(pics)
    Y = np.array(labels)
    print(X.shape)
    print(Y.shape)
    if onehot:
        Y = one_hot(Y, i)
    print(Y.shape)
    return X, Y

# data will be reshaped as (num_imgs, num_channels, height, width)
# in this case:            (50000, 3, 32, 32)
def cifar10(onehot=True):
    path = "data-cifar10"
    X = np.zeros((50000, 3072))
    Y = np.zeros((50000, 1)).astype(int)
    for i in range(1,6):
        f = open(os.path.join(path, "data_batch_" + str(i)), 'rb')
        dict = pickle.load(f, encoding="latin1")
        X[(i-1) * 10000: i * 10000,:] = dict['data']
        Y[(i-1) * 10000: i * 10000,:] = np.array(dict['labels']).reshape((10000,1)).astype(int)
    print(Y.dtype)
    Y = one_hot(Y, 10)
    print(Y.shape)
    print(Y[:10, :])
    X = X.reshape(50000, 3, 32, 32)

    print("test")
    f = open(os.path.join(path, "test_batch"), 'rb')
    dict = pickle.load(f, encoding="latin1")
    X_test = dict['data'].reshape(10000, 3, 32, 32)
    Y_test = np.array(dict['labels']).reshape((10000,1)).astype(int)
    print(X_test.shape)
    print(Y_test.shape)
    Y_test = one_hot(Y_test, 10)
    return X, Y, X_test, Y_test, 3, 32


def load_data(dataset):
    if dataset == "mnist":
        return mnist()
    if dataset == "cifar10":
        return cifar10()
    
#---------------------------------------------------------------------
def plot_filters(w, channels, idx, title):
    if channels == 1:
        plot_grey_filters(w, idx, title)
    elif channels == 3:
        plot_color_filters(w, idx, title)

def plot_grey_filters(x, idx, title=""):
    num_filters = x.shape[0]
    numrows = 10
    numcols = int(np.ceil(num_filters/10))
    plt.figure(figsize=(numrows, numcols))
    gs = gridspec.GridSpec(numcols, numrows)
    gs.update(wspace=0.1)
    gs.update(hspace=0.0)
    
    print("Plotting filters...")
    print(x.shape)
    for i in range(num_filters):
        ax = plt.subplot(gs[i])
        w = x[i, :, :, :]
        w = np.swapaxes(w, 0, 1)
        w = np.swapaxes(w, 1, 2)
        ax.imshow(w[:, :, 0], cmap=plt.cm.gist_yarg,
                  interpolation='nearest', aspect='equal')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.axis('off')
    plt.savefig(os.path.join('convfilters', title + '_' + str(idx) + '_convcaustics.png'))
    plt.close('all')

def plot_color_filters(x, idx, title=""):
    num_filters = x.shape[0]
    numrows = 10
    numcols = int(np.ceil(num_filters/10))
    plt.figure(figsize=(numrows, numcols))
    gs = gridspec.GridSpec(numcols, numrows)
    gs.update(wspace=0.1)
    gs.update(hspace=0.0)

    print("plotting color filters")
    for i in range(num_filters):
        ax = plt.subplot(gs[i])
        w = x[i, :, :, :]
        w = np.swapaxes(w, 0, 1)
        w = np.swapaxes(w, 1, 2)

        #normalize
        
        r = w[:,:,0] - np.min(w[:,:,0])
        g = w[:,:,1] - np.min(w[:,:,1])
        b = w[:,:,2] - np.min(w[:,:,2])
        r = r * 1.0 / np.max(r)
        g = g * 1.0 / np.max(g)
        b = b * 1.0 / np.max(b)
        w = np.dstack((r,g,b))
        ax.imshow(w,
                  cmap=plt.cm.gist_yarg,
                  interpolation='nearest',
                  aspect='equal')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.axis('off')
    plt.savefig(os.path.join('convfilters', title+ '_' + str(idx) + '_convcaustics.png'))
    print(os.path.join('convfilters', title+ '_' + str(idx) + '_convcaustics.png'))
    plt.close('all')

#--------------------------------------------------------------------------------------------
def plot_predictions(samples, predictions, k, batch_size, imgshape):
    samples = samples.reshape(batch_size, 3, imgshape, imgshape)
    predictions = predictions.reshape(batch_size, 3, imgshape, imgshape)
    samples = np.swapaxes(samples, 1,2)
    samples = np.swapaxes(samples, 2,3)
    predictions = np.swapaxes(predictions, 1,2)
    predictions = np.swapaxes(predictions, 2,3)
    for i in range(samples.shape[0]):
        fig = plt.figure(figsize=(5, 5))
        plt.imshow(samples[i, :, :, :],
                cmap=plt.cm.gist_yarg, interpolation='nearest',
                aspect='equal')
        path = "convpredictions"
        fname = str(k) + '_' + str(i) + 'sample_IN.png'
        plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
        plt.savefig(os.path.join(path, fname), dpi=imgshape)
        plt.close('all')
        fig = plt.figure(figsize=(5, 5))
        print(predictions[i, :, :, :].shape)
        print(predictions[i, :, :, :])
        plt.imshow(predictions[i, :, :, :],
                cmap=plt.cm.gist_yarg, interpolation='nearest',
                aspect='equal')
        fname = str(k) + '_' + str(i) + 'sample_OUT.png'
        plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
        plt.savefig(os.path.join(path, fname), dpi=imgshape)
        plt.close('all')

#--------------------------------------------------------------------
def plot_volume(volumes, k, title):
    for ind in [0,4]:
        vols = volumes[ind,:]
        vols = (vols-np.min(vols))/(np.max(vols)-np.min(vols))
        print(volumes.shape)
        numrows = 10
        numcols = int(np.ceil(vols.shape[0]/10))
        plt.figure(figsize=(numrows, numcols))
        gs = gridspec.GridSpec(numcols, numrows)
        gs.update(wspace=0.1)
        gs.update(hspace=0.0)
        
        for i in range(vols.shape[0]):
            ax = plt.subplot(gs[i])
            if i < vols.shape[0]:
                w = vols[i,:]
                
                ax.imshow(w,
                        cmap=plt.cm.gist_yarg,
                        interpolation='nearest',
                        aspect='equal')
                ax.set_xticklabels([])
                ax.set_yticklabels([])
                ax.axis('off')
        if ind==0:
            plt.savefig(os.path.join('convvolumes', title + '_' + str(k) + '_' + '_vol_A.png'))
        else:
            plt.savefig(os.path.join('convvolumes', title + '_' + str(k) + '_' + '_vol_B.png'))
        plt.close('all')

#--------------------------------------------------------------------
def plot_predictions_grid(samples, predictions, k, imgshape, title=""):
    if imgshape[1] == 1:
        predictions_grid_grey(samples, predictions, k, imgshape, title)
    elif imgshape[1] == 3:
        predictions_grid_color(samples, predictions, k, imgshape, title)
    return

def predictions_grid_color(samples, predictions, k, imgshape, title):
    batch_size = samples.shape[0]
    print("printintg predictions:")
    print(samples.shape)
    print(imgshape)
    print(predictions.shape)
    samples = samples.reshape(batch_size, imgshape[1], imgshape[2], imgshape[3])
    predictions = predictions.reshape(batch_size, imgshape[1], imgshape[2], imgshape[3])
    plt.figure(figsize=(10, 10))
    gs = gridspec.GridSpec(5, 2)
    for i in range(10):
        ax = plt.subplot(gs[i])
        if i % 2 == 0:
            w = samples[i/2, :, :, :]
        else:
            w = predictions[i/2, :, :, :]
        w = np.swapaxes(w, 0, 1)
        w = np.swapaxes(w, 1, 2)
        ax.imshow(w,
                  cmap=plt.cm.gist_yarg,
                  interpolation='nearest',
                  aspect='equal')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.axis('off')
    gs.update(wspace=0)
    plt.savefig(os.path.join('convpredictions', str(k) + '_' + title + '.png'),
                bbox_inches='tight')
    plt.close('all')


'convpredictions/0_preds.png'
def predictions_grid_grey(samples, predictions, idx, imgshape, title):
    batch_size = samples.shape[0]
    samples = samples.reshape(batch_size, imgshape[2], imgshape[3])
    predictions = predictions.reshape(batch_size, imgshape[2], imgshape[3])
    plt.figure(figsize=(6, 10))
    gs = gridspec.GridSpec(6, 2)
    for i in range(12):
        ax = plt.subplot(gs[i])
        if i % 2 == 0:
            w = samples[i/2, :, :]
        else:
            w = predictions[i/2, :, :]
            
        ax.imshow(w,
                  cmap=plt.cm.gist_yarg,
                  interpolation='nearest',
                  aspect='equal',
                  vmin=0, vmax=1)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.axis('off')

    plt.savefig('/data/fs4/home/bradh/vaeOutput/'+str(idx)+'convpredictions.png',
                bbox_inches='tight')
    #plt.savefig(os.path.join('convpredictions', str(idx) + '_' + title + '.png'),
    #            bbox_inches='tight')
    plt.close('all')

    
#-------------------------------------------------------------------

    
def plot_costs(costs):
    #print(costs)
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111)
    plt.plot(np.log(costs))
    plt.savefig('convcosts.png')
    plt.close('all')


def plot_testimg(imgs):
    for i in range(np.min([imgs.shape[0], 10])):
        img = imgs[i, :]
        #img = img.reshape(ch28, 28)
        img = np.swapaxes(img, 0, 1)
        img = np.swapaxes(img, 1, 2)
        fig = plt.figure()
        ax = fig.add_subplot(111)
        print(img.shape)
        ax.imshow(img, cmap=plt.cm.gist_yarg)#, interpolation='nearest')
    # cmap=plt.cm.gist_yarg,
    # interpolation='nearest',
    # aspect='equal')
        plt.savefig(str(i) + '_testimg.png')
        #plt.savefig(os.path.join('fanta', str(i) + '_testimg.png'))
        plt.close()

In [ ]:
trX, trY, teX, teY, channels, img_x = mnist(onehot=True)
trX = trX.reshape(trX.shape[0], 784)
teX = teX.reshape(teX.shape[0], 784)
X = T.fmatrix()
e = T.matrix()

n_code = 2

w1, b1, wmu, bmu, wsigma, bsigma, w2, b2, w3, b3 = get_params(n_code)

mu, log_sigma, z, out = model(X, w1, b1, wmu, bmu, wsigma, bsigma, w2, b2, w3, b3, e)
recon_cost = T.sum(T.sqr(X-out))
kl_cost = 0.5 * T.sum(1 + 2 * log_sigma - mu ** 2 - T.exp(2 * log_sigma))

cost = recon_cost - kl_cost

params = [w1, b1, wmu, bmu, wsigma, bsigma, w2, b2, w3, b3]
updates = RMSprop(cost, params, lr=0.001)

train = theano.function(inputs=[X, e], outputs=cost, updates=updates, allow_input_downcast=True)

reconstruct = theano.function(inputs=[X, e], outputs=[out, cost], allow_input_downcast=True)

zfan = T.fmatrix()
fantasy = generate(zfan, w2, b2, w3, b3)
fantasize = theano.function(inputs=[zfan], outputs=fantasy, allow_input_downcast=True)    

for i in range(101):
    for start, end in zip(range(0, len(trX), 128), range(128, len(trX), 128)):
        c = train(trX[start:end], floatX(np.random.randn(128, n_code)))
    
    samples = teX[:10, :]        
    r, cost = reconstruct(samples, floatX(np.random.randn(10, n_code)))
    print(str(i)+'  ', cost)
    write(str(i) + ": " + str(cost))
    plotter(samples, r, 28, i)
    
    samples = floatX(np.random.randn(20, n_code))
    f = fantasize(samples)
    f1 = f[:10, :]
    f2 = f[10:, :]
    shp = (f1.shape[0], 1, 28, 28)
    f1 = f1.reshape(shp)
    f2 = f2.reshape(shp)
    plot_predictions_grid(f1, f2, i, shp, 'generated')

In [ ]:
# Gaussian prior
def model(X, w1, b1, wmu, bmu, wsigma, bsigma, w2, b2, w3, b3, e):
    h1 = rectify(T.dot(X, w1) + b1) # encoding
    mu = T.dot(h1, wmu) + bmu # using encoding to parameterize mu
    log_sigma = 0.5 * (T.dot(h1, wsigma) + bsigma) # using enc to param sigma
    z = mu + T.exp(log_sigma) * e 
    h2 = rectify(T.dot(z, w2) + b2) # decoder
    out = rectify(T.dot(h2, w3) + b3) # decoder
    return mu, log_sigma, z, out
# need mu, log_sigma, z, wmu/bmu, wsigma/bsigma
# X is input, but you can input unif random X